# Step 1: Setup and Initialization
First, make sure you have the Earth Engine Python API installed and authenticated on your local machine. You can install it via pip and authenticate using the following commands:

In [ ]:
# !pip3 install earthengine-api
# !pip3 install folium
# earthengine authenticate

In [ ]:
# ALL
import ee
import folium
import PIL
import urllib

# AS
import matplotlib.pyplot as plt
import numpy as np

# FROM
from IPython.display import Image

# Step 2: Define Area of Interest
Specify the geographic boundaries of your area of interest (AOI) in Sudan. This could be a specific region where you suspect crop diseases. The coordinates are defined in longitude and latitude.

In [ ]:
# Authenticate the Earth Engine module.
ee.Authenticate()

In [ ]:
# Initialize the Earth Engine library
ee.Initialize()

# Step 2: Define Area of Interest
Specify the geographic boundaries of your area of interest (AOI) in Sudan. This could be a specific region where you suspect crop diseases. The coordinates are defined in longitude and latitude.

In [ ]:
# Define a Point and then buffer it to create an area (adjust buffer size as needed).
aoi = ee.Geometry.Point([30.5, 12.0]).buffer(10000)  # Buffer in meters around the point

# Define the Study Area in Sudan
sudan_region = ee.Geometry.Rectangle([32.5, 13.0, 34.0, 14.5])  # Adjust these coordinates as needed

In [ ]:
# Step 2: Define Training Data
# Note: Adjust the paths according to your actual GEE assets for Sudan training data
sudan_training = ee.FeatureCollection('path_to_sudan_training_data/cropland').merge(
    ee.FeatureCollection('path_to_sudan_training_data/fallow_land')).merge(
    ee.FeatureCollection('path_to_sudan_training_data/irrigated_land'))

# Step 3: Acquire Sentinel-2 Imagery
Fetch Sentinel-2 imagery that covers your AOI. Filter by date and other relevant properties like cloud cover.

In [ ]:
# Set date range for historical imagery relevant to your analysis
start_date = '2022-01-01' # Start of growing season
end_date = '2022-12-31' # End of growing season

In [ ]:
# Load Sentinel-2 imagery and filter by date, cloud cover and area
s2 = (ee.ImageCollection('COPERNICUS/S2')
      .filterDate(start_date, end_date)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Low cloud cover
      .filterBounds(sudan_region))

In [ ]:
# Use median reduction to combine images into one representative image
composite = s2.median().clip(aoi)

# Step 4: Preprocess the Data
Select specific bands and perform any necessary preprocessing like normalization or spectral indices (e.g., NDVI for vegetation health).

In [ ]:
# Select bands typically useful for vegetation analysis (Red, Green, Blue, NIR)
bands = ['B4', 'B3', 'B2', 'B8']
image = composite.select(bands)

# Select the least cloudy image and calculate NDVI.
least_cloudy = s2.sort('CLOUDY_PIXEL_PERCENTAGE').first()

# Calculate NDVI (Normalized Difference Vegetation Index)
# ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi = least_cloudy.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [ ]:
# Function to add EE layer to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

In [ ]:
# Function to process the imagery and classify land use
def land_map():
    # Select the least cloudy image
    least_cloudy = s2.sort('CLOUDY_PIXEL_PERCENTAGE').first()
    ndvi = least_cloudy.normalizedDifference(['B8', 'B4']).rename('NDVI')
    image_with_indices = least_cloudy.addBands(ndvi)
    
    # Sampling the image to get training data
    training = image_with_indices.sampleRegions(
        collection=sudan_training,
        properties=['landcover'],
        scale=10
    )
    
    # Train a classifier
    classifier = ee.Classifier.smileRandomForest(50).train(
        features=training,
        classProperty='landcover',
        inputProperties=image_with_indices.bandNames()
    )
    
    # Classify the image
    classified_image = image_with_indices.classify(classifier)
    
    # Add layers to the map
    map = ee.Map()
    map.centerObject(sudan_region, 8)
    map.addLayer(classified_image, {'min': 0, 'max': 3, 'palette': ['green', 'yellow', 'blue']}, 'Land Cover')
    map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')
    
    return map


# Step 5: Visualization
Visualize the data to manually inspect the area for signs of crop disease or to validate outputs.

In [ ]:
# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[13.75, 33.25], zoom_start=8, height=500)

# Add the NDVI layer to the map.
my_map.add_ee_layer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
my_map

In [ ]:
# To visualize the result in a Jupyter Notebook:
# url = land_map().getMapId()['tile_fetcher'].url_format
# Image(url=url)

In [ ]:
# Fetch the NDVI image data as a NumPy array for visualization
url = ndvi.getThumbURL({'min': 0, 'max': 1, 'dimensions': 512, 'format': 'png'})
# Example code to display URL (actual code will depend on your environment)


In [ ]:
# Convert the image to RGB format
image_rgb = np.array(PIL.Image.open(urllib.request.urlopen(url)).convert('RGB'))

# Display the RGB image
plt.imshow(image_rgb)

In [ ]:
import ee
import time

ee.Initialize()

# Define the region of interest (ROI) using a polygon
roi = ee.Geometry.Polygon(
    [
        [
            [33.361, 14.541],  # Northwestern boundary
            [33.661, 14.542],  # Northeastern boundary
            [33.660, 14.320],  # Southeastern boundary
            [33.360, 14.321]   # Southwestern boundary
        ]
    ], None, False)


# Define the Sentinel-2 Surface Reflectance image collection
sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR")

# Set variables
start_date = ee.Date('2021-02-01')
end_date = ee.Date('2021-05-20')
cloud_percentage = 30
interval = 30
increment = 'day'

# Get district of interest and center map view (Assuming `roi` is defined)
# Map.centerObject(roi, 8)  # Comment this line if not using an interactive map environment like in Python

# Function to add NDVI to each image in the collection
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def add_indices(image):
    # Calculate NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    # Calculate EVI using the formula: 2.5 * (NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1)
    evi = image.expression(
        '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    
    # Add both NDVI and EVI as bands
    return image.addBands([ndvi, evi])

# Get the image collection
sentinel_collection = ee.ImageCollection('COPERNICUS/S2')  # Assumed Sentinel-2 image collection ID

# Use the updated function to map over the image collection
sentinel_ts = (sentinel_collection 
               .filterBounds(roi) 
               .filterDate(start_date, end_date) 
               .select(['B4', 'B8', 'B2']) 
               .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) 
               .map(add_indices) 
               .select(['NDVI', 'EVI'])
               )

# Make a list of start times for composites
second_date = start_date.advance(interval, increment)
increase = second_date.millis().subtract(start_date.millis())
date_list = ee.List.sequence(start_date.millis(), end_date.millis(), interval * 8640000) # increase)
# date_list = ee.List.sequence(start_date.millis(), end_date.millis(), ee.Date(interval, increment).millis().subtract(start_date.millis()))

# Ensure there are images in the collection
print("Number of images in collection: ", sentinel_ts.size().getInfo())
print("Date list: ", len(date_list.getInfo()))

# Monthly composite
def create_composite(date):
    date = ee.Date(date)
    filtered_collection = sentinel_ts.filterDate(date, date.advance(interval, increment))
    return filtered_collection.mean().set('system:time_start', date.millis())

new_col = ee.ImageCollection(date_list.map(create_composite))

# Convert collection to image
new_col = new_col.toBands() # .clip(roi)

print("New collection: ", len(new_col.getInfo()))

# Visualize NDVI as red, EVI as yellow, and use the 'B4' band as green
visual_params = {
    'min': 0,
    'max': 1,
    'bands': ['0_NDVI','1_NDVI', '2_NDVI'],
    'palette': ['blue', 'white', 'green'],
    # 'region': roi.coordinates().getInfo()  # Explicitly get coordinates
}


# Calculate NDVI and NDRE
# ndvi = sentinel_ts.normalizedDifference(['B8', 'B4']).rename('NDVI')
# ndre = sentinel_ts.normalizedDifference(['B8', 'B5']).rename('NDRE')

# ndre_url = ndre.getThumbURL(visual_params)

# Generate thumbnail URL for the NDVI composite
ndvi_url = new_col.visualize(**visual_params).getThumbURL({
    # 'min': 0,
    # 'max': 1,
    'dimensions': 1200,
    'region': roi.getInfo()['coordinates']
})

# ndre_url = new_col.visualize(**visual_params).getThumbURL({
#     # 'min': 0,
#     # 'max': 1,
#     'dimensions': 1200,
#     'region': roi.getInfo()['coordinates']
# })

print('NDVI image URL:', ndvi_url)
# print('NDRE image URL:', ndre_url)

# Define export parameters for Google Drive
task = ee.batch.Export.image.toDrive(
    image=new_col.visualize(**visual_params),
    description='Local_NDVI_Composite',
    folder='Multimodal Cash Crop Disease Detection - Gezira Scheme',  # Make sure this folder exists in your Google Drive
    # fileNamePrefix=f'Local_NDVI_Composite_',
    scale=30,  # Scale is set to 30 meters
    region=roi.coordinates().getInfo(),
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

# Optionally, print the URL (if small region) or monitor status
while task.status()['state'] in ['READY', 'RUNNING']:
    print(task.status())
    time.sleep(10)
else:
    print('Export completed with state:', task.status()['state'])